In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

In [2]:
pip install MiniSom

  Created wheel for MiniSom: filename=MiniSom-2.2.3-cp36-none-any.whl size=7550 sha256=13f243081cfcf9070e243a9f501bf94c08c52e7a5ff5156ed2b875a4a9b038a4
  Stored in directory: /root/.cache/pip/wheels/35/62/79/4d921062c847ee15a3bb3ac2ea984ed401c8b6b2944f08f697
Successfully built MiniSom


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from datetime import datetime

from minisom import MiniSom
import matplotlib.pyplot as plt

In [0]:
class Data:
    # Load Adult dataset and seperate to features(X) and target(y)
    def __init__(self, path='adult.csv'):
        df = shuffle(pd.read_csv(path, engine='python'))
        df = self.clean(df)

        self.y = df.pop('income')
        self.X = df

        # Label encode y
        self.y_encoder = LabelEncoder()
        self.y = self.y_encoder.fit_transform(self.y)

        # One Hot encode X
        self.X = pd.get_dummies(self.X)

        for name in self.X.columns:
            if self.X[name].dtype == 'object':
                self.X[name] = self.X[name].astype('category')

    def clean(self, df):
        return df.replace('?', np.nan).dropna().drop('fnlwgt', axis=1)

    def train_test_split(self):
        X_train, X_test, y_train, y_test = train_test_split(
            self.X, self.y, test_size=0.2)
        y_train = pd.Series(y_train, index=X_train.index)
        y_test = pd.Series(y_test, index=X_test.index)
        return (X_train, X_test, y_train, y_test)

In [5]:
som_start=datetime.now()

data = Data()

quasi_identifiers = ['age', 'educational-num',
                         'capital-gain', 'capital-loss', 'hours-per-week']
features = data.X[quasi_identifiers]
target = data.y

#要調整
width=150
height=150
sigma=.9
lr=.2
epochs=1e5
#epochs = 100

verbose=True
log = 1000

som = MiniSom(width, height, features.shape[1], sigma, lr)
som.train_random(features.values, int(epochs), verbose=True)

out = []
for step, (X, y) in enumerate(zip(features.values, target)):
    new_X = som.winner(X)
    out.append((new_X, X, y))
    if(verbose == True and step % log == 0):
        print(f'*Creating SOM: [{step}/{features.shape[0]}]')
som_data = np.array(out)

new_data = []
new_X = []
X = []
y = []
for i in range(0, len(som_data[:,0])):
    new_X.append(np.asarray(som_data[:,0][i]))
    X.append(np.asarray(som_data[:,1][i]))
    y.append(np.asarray(som_data[:,2][i]))
new_data = (new_X, X, y)

print("Time required for SOM: " + str(datetime.now()-som_start))

new_data

 [ 100000 / 100000 ] 100% - 0:00:00 left 
 quantization error: 157.9368216980081
 topographic error: 0.1457255318207952
*Creating SOM: [0/45222]
*Creating SOM: [1000/45222]
*Creating SOM: [2000/45222]
*Creating SOM: [3000/45222]
*Creating SOM: [4000/45222]
*Creating SOM: [5000/45222]
*Creating SOM: [6000/45222]
*Creating SOM: [7000/45222]
*Creating SOM: [8000/45222]
*Creating SOM: [9000/45222]
*Creating SOM: [10000/45222]
*Creating SOM: [11000/45222]
*Creating SOM: [12000/45222]
*Creating SOM: [13000/45222]
*Creating SOM: [14000/45222]
*Creating SOM: [15000/45222]
*Creating SOM: [16000/45222]
*Creating SOM: [17000/45222]
*Creating SOM: [18000/45222]
*Creating SOM: [19000/45222]
*Creating SOM: [20000/45222]
*Creating SOM: [21000/45222]
*Creating SOM: [22000/45222]
*Creating SOM: [23000/45222]
*Creating SOM: [24000/45222]
*Creating SOM: [25000/45222]
*Creating SOM: [26000/45222]
*Creating SOM: [27000/45222]
*Creating SOM: [28000/45222]
*Creating SOM: [29000/45222]
*Creating SOM: [30000/4

([array([116,   1]),
  array([90,  0]),
  array([115,   2]),
  array([109,   0]),
  array([44,  3]),
  array([92, 18]),
  array([48,  6]),
  array([105,  21]),
  array([112,  16]),
  array([99, 17]),
  array([93, 13]),
  array([100,   5]),
  array([113,   3]),
  array([103,  21]),
  array([103,   6]),
  array([108,  11]),
  array([109,   6]),
  array([ 55, 102]),
  array([110,  22]),
  array([104,  21]),
  array([113,   3]),
  array([80,  7]),
  array([117,  10]),
  array([53,  5]),
  array([108,   3]),
  array([44, 99]),
  array([123,  11]),
  array([93,  7]),
  array([116,   9]),
  array([107,  21]),
  array([102,  18]),
  array([105,  12]),
  array([86,  6]),
  array([105,   1]),
  array([97, 20]),
  array([96,  4]),
  array([97, 16]),
  array([102,   7]),
  array([93,  4]),
  array([94,  9]),
  array([96, 16]),
  array([122,   8]),
  array([110,   0]),
  array([112,   1]),
  array([104,  26]),
  array([101,  17]),
  array([105,   4]),
  array([108,  19]),
  array([87, 17]),
  array

In [0]:
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from collections import Counter, defaultdict

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

In [0]:
class TrainingModel:
    def __init__(self, input_shape):
        self.model = Sequential()
        self.model.add(Dense(64, activation='relu', input_shape=input_shape))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

    def fit(self, data, label):
        self.model.fit(data, label, epochs=1, batch_size=128, verbose=0)

    def predict(self, data):
        return self.model.predict_classes(data)
    
    def evaluate(self, X_test, y_test, print_report=True):
        y_predicted = self.predict(X_test)
        y_predicted_probs = self.model.predict_proba(X_test)
        if print_report:
            self.print_report(y_test, y_predicted, y_predicted_probs)
        else:
            accuracy = accuracy_score(y_test, y_predicted)
            report = classification_report(y_test, y_predicted, output_dict=True)
            auc_score = roc_auc_score(y_test, y_predicted_probs)
            matrix = confusion_matrix(y_test, y_predicted)

            return {
                'accuracy': accuracy,
                'auc_score': auc_score,
                **report['weighted avg'],
            }

    def print_report(self, test, predicted, predicted_probs):
        accuracy = accuracy_score(test, predicted)
        report = classification_report(test, predicted)
        matrix = confusion_matrix(test, predicted)

        print('Accuracy score: {:.5f}'.format(accuracy))
        print('-' * 20)
        print('Confusion Matrix:')
        print(matrix)
        print('-' * 20)
        print(report)
        print('-' * 20)
        print('AUC score: {:.5f}'.format(roc_auc_score(test, predicted_probs)))

In [12]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 50
K = int(len(new_data[0])//(k*2))
D = 500//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=452. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=452. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=452. Setting it to 3*k
  init_size=init_size)


K=452
cluster 0 has 109 data points
cluster 1 has 71 data points
cluster 2 has 105 data points
cluster 3 has 94 data points
cluster 4 has 226 data points
cluster 5 has 137 data points
cluster 6 has 172 data points
cluster 7 has 157 data points
cluster 8 has 214 data points
cluster 9 has 71 data points
cluster 10 has 35 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=442. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=442. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=442. Setting it to 3*k
  init_size=init_size)


K=442
cluster 0 has 105 data points
cluster 1 has 154 data points
cluster 2 has 293 data points
cluster 3 has 164 data points
cluster 4 has 131 data points
cluster 5 has 82 data points
cluster 6 has 49 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=432. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=432. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=432. Setting it to 3*k
  init_size=init_size)


...

K=322
cluster 0 has 71 data points
cluster 1 has 0 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=312. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=312. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=312. Setting it to 3*k
  init_size=init_size)


K=312
cluster 0 has 197 data points
cluster 1 has 261 data points
cluster 2 has 117 data points
cluster 3 has 238 data points
cluster 4 has 236 data points
cluster 5 has 278 data points
cluster 6 has 253 data points
cluster 7 has 78 data points
cluster 8 has 47 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)


K=302
cluster 0 has 51 data points
cluster 1 has 207 data points
cluster 2 has 170 data points
cluster 3 has 162 data points
cluster 4 has 212 data points
cluster 5 has 333 data points
cluster 6 has 70 data points
cluster 7 has 223 data points
cluster 8 has 271 data points
cluster 9 has 84 data points
cluster 10 has 184 data points
cluster 11 has 89 data points
cluster 12 has 46 data points
K=292
cluster 0 has 178 data points
cluster 1 has 146 data points
cluster 2 has 50 data points
cluster 3 has 210 data points
cluster 4 has 137 data points
cluster 5 has 275 data points
cluster 6 has 379 data points
cluster 7 has 108 data points
cluster 8 has 107 data points
cluster 9 has 172 data points
cluster 10 has 66 data points
cluster 11 has 288 data points
cluster 12 has 128 data points
cluster 13 has 218 data points
cluster 14 has 141 data points
cluster 15 has 306 data points
cluster 16 has 74 data points
cluster 17 has 140 data points
cluster 18 has 113 data points
cluster 19 has 105 data 

In [13]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready50 = data_sorted.drop('cluster', axis=1)
print(data_ready50)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
17941  55.8105         8.93684            0            0        52.4105      0
43576  55.8105         8.93684            0            0        52.4105      1
26254  55.8105         8.93684            0            0        52.4105      1
790    55.8105         8.93684            0            0        52.4105      0
1078   55.8105         8.93684            0            0        52.4105      1
...        ...             ...          ...          ...            ...    ...
9547   35.2595         9.64324            0            0        45.4649      0
21823  35.2595         9.64324            0            0        45.4649      1
8030   35.2595         9.64324            0            0        45.4649      0
34910  35.2595         9.64324            0            0        45.4649      1
34130  35.2595         9.64324            0            0        45.4649      1

[45222 rows x 6 columns]
Time required for perturba

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data_ready50.iloc[:,0:5], np.array(data_ready50.iloc[:,5:], dtype=int), test_size=0.2)
model50 = TrainingModel((5,))
model50.fit(X_train, y_train)
model50.evaluate(X_test, y_test, print_report=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Accuracy score: 0.77557
--------------------
Confusion Matrix:
[[6332  481]
 [1549  683]]
--------------------
              precision    recall  f1-score   support

           0       0.80      0.93      0.86      6813
           1       0.59      0.31      0.40      2232

    accuracy                           0.78      9045
   macro avg       0.70      0.62      0.63      9045
weighted avg       0.75      0.78      0.75      9045

--------------------
AUC score: 0.65047


In [15]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 30
K = int(len(new_data[0])//(k*2))
D = 500//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=753. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=753. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=753. Setting it to 3*k
  init_size=init_size)


K=753
cluster 0 has 82 data points
cluster 1 has 237 data points
cluster 2 has 14 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=737. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=737. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=737. Setting it to 3*k
  init_size=init_size)


K=737
cluster 0 has 188 data points
cluster 1 has 82 data points
cluster 2 has 2 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=721. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=721. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=721. Setting it to 3*k
  init_size=init_size)


...

K=337
cluster 0 has 117 data points
cluster 1 has 322 data points
cluster 2 has 118 data points
cluster 3 has 98 data points
cluster 4 has 235 data points
cluster 5 has 168 data points
cluster 6 has 192 data points
cluster 7 has 65 data points
cluster 8 has 141 data points
cluster 9 has 175 data points
cluster 10 has 125 data points
cluster 11 has 226 data points
cluster 12 has 299 data points
cluster 13 has 131 data points
cluster 14 has 222 data points
cluster 15 has 112 data points
cluster 16 has 183 data points
cluster 17 has 187 data points
cluster 18 has 164 data points
cluster 19 has 210 data points
cluster 20 has 69 data points
cluster 21 has 157 data points
cluster 22 has 125 data points
cluster 23 has 137 data points
cluster 24 has 176 data points
cluster 25 has 225 data points
cluster 26 has 225 data points
cluster 27 has 227 data points
cluster 28 has 179 data points
cluster 29 has 66 data points
cluster 30 has 234 data points
cluster 31 has 171 data points
cluster 32 has 2

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=321. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=321. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=321. Setting it to 3*k
  init_size=init_size)


K=321
cluster 0 has 295 data points
cluster 1 has 253 data points
cluster 2 has 144 data points
cluster 3 has 110 data points
cluster 4 has 300 data points
cluster 5 has 136 data points
cluster 6 has 91 data points
cluster 7 has 318 data points
cluster 8 has 71 data points
cluster 9 has 157 data points
cluster 10 has 140 data points
cluster 11 has 283 data points
cluster 12 has 170 data points
cluster 13 has 175 data points
cluster 14 has 189 data points
cluster 15 has 140 data points
cluster 16 has 63 data points
cluster 17 has 42 data points
cluster 18 has 170 data points
cluster 19 has 77 data points
cluster 20 has 167 data points
cluster 21 has 131 data points
cluster 22 has 455 data points
cluster 23 has 100 data points
cluster 24 has 189 data points
cluster 25 has 105 data points
cluster 26 has 167 data points
cluster 27 has 217 data points
cluster 28 has 101 data points
cluster 29 has 0 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)


K=305
cluster 0 has 251 data points
cluster 1 has 123 data points
cluster 2 has 68 data points
cluster 3 has 267 data points
cluster 4 has 230 data points
cluster 5 has 144 data points
cluster 6 has 122 data points
cluster 7 has 397 data points
cluster 8 has 113 data points
cluster 9 has 214 data points
cluster 10 has 364 data points
cluster 11 has 183 data points
cluster 12 has 172 data points
cluster 13 has 121 data points
cluster 14 has 173 data points
cluster 15 has 84 data points
cluster 16 has 72 data points
cluster 17 has 234 data points
cluster 18 has 104 data points
cluster 19 has 126 data points
cluster 20 has 139 data points
cluster 21 has 112 data points
cluster 22 has 115 data points
cluster 23 has 69 data points
cluster 24 has 192 data points
cluster 25 has 79 data points
cluster 26 has 194 data points
cluster 27 has 71 data points
cluster 28 has 147 data points
cluster 29 has 39 data points
cluster 30 has 180 data points
cluster 31 has 345 data points
cluster 32 has 184 

In [16]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready30 = data_sorted.drop('cluster', axis=1)
print(data_ready30)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
38619  24.6385         9.84615            0            0        60.0154      0
32722  24.6385         9.84615            0            0        60.0154      0
14008  24.6385         9.84615            0            0        60.0154      0
5020   24.6385         9.84615            0            0        60.0154      0
39429  24.6385         9.84615            0            0        60.0154      1
...        ...             ...          ...          ...            ...    ...
10896  17.3475         6.63559            0            0        23.2203      0
43620  17.3475         6.63559            0            0        23.2203      0
18082  17.3475         6.63559            0            0        23.2203      0
24233  17.3475         6.63559            0            0        23.2203      0
9569   17.3475         6.63559            0            0        23.2203      0

[45222 rows x 6 columns]
Time required for perturba

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data_ready30.iloc[:,0:5], np.array(data_ready30.iloc[:,5:], dtype=int), test_size=0.2)
model30 = TrainingModel((5,))
model30.fit(X_train, y_train)
model30.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.78076
--------------------
Confusion Matrix:
[[6347  456]
 [1527  715]]
--------------------
              precision    recall  f1-score   support

           0       0.81      0.93      0.86      6803
           1       0.61      0.32      0.42      2242

    accuracy                           0.78      9045
   macro avg       0.71      0.63      0.64      9045
weighted avg       0.76      0.78      0.75      9045

--------------------
AUC score: 0.67798


In [18]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 20
K = int(len(new_data[0])//(k*2))
D = 500//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1130. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1130. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1130. Setting it to 3*k
  init_size=init_size)


K=1130
cluster 0 has 25 data points
cluster 1 has 3 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1105. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1105. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1105. Setting it to 3*k
  init_size=init_size)


K=1105
cluster 0 has 112 data points
cluster 1 has 232 data points
cluster 2 has 61 data points
cluster 3 has 21 data points
cluster 4 has 226 data points
cluster 5 has 67 data points
cluster 6 has 27 data points
cluster 7 has 55 data points
cluster 8 has 45 data points
cluster 9 has 98 data points
cluster 10 has 10 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1080. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1080. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1080. Setting it to 3*k
  init_size=init_size)


...

K=355
cluster 0 has 266 data points
cluster 1 has 149 data points
cluster 2 has 73 data points
cluster 3 has 128 data points
cluster 4 has 100 data points
cluster 5 has 146 data points
cluster 6 has 64 data points
cluster 7 has 179 data points
cluster 8 has 319 data points
cluster 9 has 172 data points
cluster 10 has 215 data points
cluster 11 has 23 data points
cluster 12 has 128 data points
cluster 13 has 89 data points
cluster 14 has 51 data points
cluster 15 has 105 data points
cluster 16 has 336 data points
cluster 17 has 144 data points
cluster 18 has 75 data points
cluster 19 has 79 data points
cluster 20 has 174 data points
cluster 21 has 170 data points
cluster 22 has 165 data points
cluster 23 has 100 data points
cluster 24 has 138 data points
cluster 25 has 251 data points
cluster 26 has 132 data points
cluster 27 has 24 data points
cluster 28 has 39 data points
cluster 29 has 101 data points
cluster 30 has 56 data points
cluster 31 has 152 data points
cluster 32 has 160 dat

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=330. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=330. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=330. Setting it to 3*k
  init_size=init_size)


K=330
cluster 0 has 225 data points
cluster 1 has 166 data points
cluster 2 has 15 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)


K=305
cluster 0 has 312 data points
cluster 1 has 259 data points
cluster 2 has 129 data points
cluster 3 has 151 data points
cluster 4 has 280 data points
cluster 5 has 275 data points
cluster 6 has 148 data points
cluster 7 has 188 data points
cluster 8 has 133 data points
cluster 9 has 127 data points
cluster 10 has 84 data points
cluster 11 has 269 data points
cluster 12 has 391 data points
cluster 13 has 157 data points
cluster 14 has 160 data points
cluster 15 has 278 data points
cluster 16 has 180 data points
cluster 17 has 183 data points
cluster 18 has 146 data points
cluster 19 has 335 data points
cluster 20 has 90 data points
cluster 21 has 134 data points
cluster 22 has 169 data points
cluster 23 has 104 data points
cluster 24 has 147 data points
cluster 25 has 183 data points
cluster 26 has 192 data points
cluster 27 has 365 data points
cluster 28 has 189 data points
cluster 29 has 78 data points
cluster 30 has 138 data points
cluster 31 has 156 data points
cluster 32 has 

In [19]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready20 = data_sorted.drop('cluster', axis=1)
print(data_ready20)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
1933      35.8         9.56296            0            0        45.3185      0
10139     35.8         9.56296            0            0        45.3185      1
5628      35.8         9.56296            0            0        45.3185      0
4250      35.8         9.56296            0            0        45.3185      0
37304     35.8         9.56296            0            0        45.3185      1
...        ...             ...          ...          ...            ...    ...
16314  49.8641         10.0777            0            0         28.835      0
23732  49.8641         10.0777            0            0         28.835      0
23758  49.8641         10.0777            0            0         28.835      1
339    49.8641         10.0777            0            0         28.835      0
14506  49.8641         10.0777            0            0         28.835      1

[45222 rows x 6 columns]
Time required for perturba

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data_ready20.iloc[:,0:5], np.array(data_ready20.iloc[:,5:], dtype=int), test_size=0.2)
model20 = TrainingModel((5,))
model20.fit(X_train, y_train)
model20.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77026
--------------------
Confusion Matrix:
[[6291  472]
 [1606  676]]
--------------------
              precision    recall  f1-score   support

           0       0.80      0.93      0.86      6763
           1       0.59      0.30      0.39      2282

    accuracy                           0.77      9045
   macro avg       0.69      0.61      0.63      9045
weighted avg       0.74      0.77      0.74      9045

--------------------
AUC score: 0.64787


In [21]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 15
K = int(len(new_data[0])//(k*2))
D = 500//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1507. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1507. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1507. Setting it to 3*k
  init_size=init_size)


K=1507
cluster 0 has 107 data points
cluster 1 has 10 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1474. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1474. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1474. Setting it to 3*k
  init_size=init_size)


K=1474
cluster 0 has 39 data points
cluster 1 has 176 data points
cluster 2 has 0 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1441. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1441. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1441. Setting it to 3*k
  init_size=init_size)


...

K=385
cluster 0 has 79 data points
cluster 1 has 119 data points
cluster 2 has 54 data points
cluster 3 has 157 data points
cluster 4 has 200 data points
cluster 5 has 166 data points
cluster 6 has 160 data points
cluster 7 has 135 data points
cluster 8 has 168 data points
cluster 9 has 203 data points
cluster 10 has 210 data points
cluster 11 has 97 data points
cluster 12 has 187 data points
cluster 13 has 52 data points
cluster 14 has 188 data points
cluster 15 has 169 data points
cluster 16 has 106 data points
cluster 17 has 74 data points
cluster 18 has 193 data points
cluster 19 has 55 data points
cluster 20 has 134 data points
cluster 21 has 65 data points
cluster 22 has 184 data points
cluster 23 has 479 data points
cluster 24 has 133 data points
cluster 25 has 112 data points
cluster 26 has 235 data points
cluster 27 has 54 data points
cluster 28 has 101 data points
cluster 29 has 117 data points
cluster 30 has 104 data points
cluster 31 has 87 data points
cluster 32 has 118 da

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=352. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=352. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=352. Setting it to 3*k
  init_size=init_size)


K=352
cluster 0 has 76 data points
cluster 1 has 253 data points
cluster 2 has 104 data points
cluster 3 has 84 data points
cluster 4 has 193 data points
cluster 5 has 333 data points
cluster 6 has 84 data points
cluster 7 has 239 data points
cluster 8 has 267 data points
cluster 9 has 14 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=319. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=319. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=319. Setting it to 3*k
  init_size=init_size)


K=319
cluster 0 has 233 data points
cluster 1 has 169 data points
cluster 2 has 65 data points
cluster 3 has 85 data points
cluster 4 has 127 data points
cluster 5 has 272 data points
cluster 6 has 94 data points
cluster 7 has 75 data points
cluster 8 has 92 data points
cluster 9 has 356 data points
cluster 10 has 51 data points
cluster 11 has 137 data points
cluster 12 has 40 data points
cluster 13 has 281 data points
cluster 14 has 234 data points
cluster 15 has 350 data points
cluster 16 has 142 data points
cluster 17 has 277 data points
cluster 18 has 140 data points
cluster 19 has 69 data points
cluster 20 has 212 data points
cluster 21 has 152 data points
cluster 22 has 333 data points
cluster 23 has 477 data points
cluster 24 has 301 data points
cluster 25 has 262 data points
cluster 26 has 0 data points
K=286
cluster 0 has 295 data points
cluster 1 has 192 data points
cluster 2 has 153 data points
cluster 3 has 145 data points
cluster 4 has 157 data points
cluster 5 has 146 dat

In [22]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready15 = data_sorted.drop('cluster', axis=1)
print(data_ready15)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

          age educational-num capital-gain capital-loss hours-per-week target
27897  38.374         10.1339            0            0        40.0157      0
1717   38.374         10.1339            0            0        40.0157      0
23376  38.374         10.1339            0            0        40.0157      0
21979  38.374         10.1339            0            0        40.0157      0
5893   38.374         10.1339            0            0        40.0157      0
...       ...             ...          ...          ...            ...    ...
7499       46         10.2385            0            0        39.9846      1
20848      46         10.2385            0            0        39.9846      1
25466      46         10.2385            0            0        39.9846      0
18627      46         10.2385            0            0        39.9846      0
42496      46         10.2385            0            0        39.9846      0

[45222 rows x 6 columns]
Time required for perturbation: 0:23:4

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data_ready15.iloc[:,0:5], np.array(data_ready15.iloc[:,5:], dtype=int), test_size=0.2)
model15 = TrainingModel((5,))
model15.fit(X_train, y_train)
model15.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.76949
--------------------
Confusion Matrix:
[[6489  276]
 [1809  471]]
--------------------
              precision    recall  f1-score   support

           0       0.78      0.96      0.86      6765
           1       0.63      0.21      0.31      2280

    accuracy                           0.77      9045
   macro avg       0.71      0.58      0.59      9045
weighted avg       0.74      0.77      0.72      9045

--------------------
AUC score: 0.44350


In [24]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 10
K = int(len(new_data[0])//(k*2))
D = 500//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2261. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2261. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2261. Setting it to 3*k
  init_size=init_size)


K=2261
cluster 0 has 127 data points
cluster 1 has 137 data points
cluster 2 has 17 data points
cluster 3 has 69 data points
cluster 4 has 98 data points
cluster 5 has 143 data points
cluster 6 has 209 data points
cluster 7 has 391 data points
cluster 8 has 21 data points
cluster 9 has 193 data points
cluster 10 has 67 data points
cluster 11 has 31 data points
cluster 12 has 99 data points
cluster 13 has 23 data points
cluster 14 has 57 data points
cluster 15 has 78 data points
cluster 16 has 70 data points
cluster 17 has 176 data points
cluster 18 has 26 data points
cluster 19 has 45 data points
cluster 20 has 46 data points
cluster 21 has 152 data points
cluster 22 has 55 data points
cluster 23 has 53 data points
cluster 24 has 124 data points
cluster 25 has 50 data points
cluster 26 has 47 data points
cluster 27 has 56 data points
cluster 28 has 54 data points
cluster 29 has 105 data points
cluster 30 has 87 data points
cluster 31 has 74 data points
cluster 32 has 11 data points
clu

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2211. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2211. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2211. Setting it to 3*k
  init_size=init_size)


K=2211
cluster 0 has 46 data points
cluster 1 has 229 data points
cluster 2 has 209 data points
cluster 3 has 49 data points
cluster 4 has 94 data points
cluster 5 has 39 data points
cluster 6 has 193 data points
cluster 7 has 41 data points
cluster 8 has 137 data points
cluster 9 has 31 data points
cluster 10 has 107 data points
cluster 11 has 391 data points
cluster 12 has 56 data points
cluster 13 has 10 data points
cluster 14 has 42 data points
cluster 15 has 182 data points
cluster 16 has 31 data points
cluster 17 has 33 data points
cluster 18 has 42 data points
cluster 19 has 146 data points
cluster 20 has 22 data points
cluster 21 has 25 data points
cluster 22 has 107 data points
cluster 23 has 172 data points
cluster 24 has 61 data points
cluster 25 has 36 data points
cluster 26 has 47 data points
cluster 27 has 72 data points
cluster 28 has 16 data points
cluster 29 has 6 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2161. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2161. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2161. Setting it to 3*k
  init_size=init_size)


...

K=411
cluster 0 has 311 data points
cluster 1 has 232 data points
cluster 2 has 159 data points
cluster 3 has 147 data points
cluster 4 has 44 data points
cluster 5 has 126 data points
cluster 6 has 128 data points
cluster 7 has 74 data points
cluster 8 has 75 data points
cluster 9 has 76 data points
cluster 10 has 226 data points
cluster 11 has 133 data points
cluster 12 has 184 data points
cluster 13 has 31 data points
cluster 14 has 285 data points
cluster 15 has 153 data points
cluster 16 has 56 data points
cluster 17 has 73 data points
cluster 18 has 171 data points
cluster 19 has 72 data points
cluster 20 has 54 data points
cluster 21 has 66 data points
cluster 22 has 209 data points
cluster 23 has 98 data points
cluster 24 has 49 data points
cluster 25 has 235 data points
cluster 26 has 174 data points
cluster 27 has 140 data points
cluster 28 has 118 data points
cluster 29 has 78 data points
cluster 30 has 214 data points
cluster 31 has 98 data points
cluster 32 has 176 data po

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=361. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=361. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=361. Setting it to 3*k
  init_size=init_size)


K=361
cluster 0 has 144 data points
cluster 1 has 342 data points
cluster 2 has 107 data points
cluster 3 has 164 data points
cluster 4 has 172 data points
cluster 5 has 236 data points
cluster 6 has 112 data points
cluster 7 has 10 data points
cluster 8 has 35 data points
cluster 9 has 108 data points
cluster 10 has 165 data points
cluster 11 has 104 data points
cluster 12 has 427 data points
cluster 13 has 110 data points
cluster 14 has 116 data points
cluster 15 has 290 data points
cluster 16 has 76 data points
cluster 17 has 25 data points
cluster 18 has 87 data points
cluster 19 has 76 data points
cluster 20 has 239 data points
cluster 21 has 287 data points
cluster 22 has 131 data points
cluster 23 has 135 data points
cluster 24 has 166 data points
cluster 25 has 144 data points
cluster 26 has 147 data points
cluster 27 has 57 data points
cluster 28 has 219 data points
cluster 29 has 61 data points
cluster 30 has 81 data points
cluster 31 has 159 data points
cluster 32 has 63 dat

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=311. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=311. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=311. Setting it to 3*k
  init_size=init_size)


K=311
cluster 0 has 230 data points
cluster 1 has 254 data points
cluster 2 has 369 data points
cluster 3 has 157 data points
cluster 4 has 268 data points
cluster 5 has 203 data points
cluster 6 has 186 data points
cluster 7 has 135 data points
cluster 8 has 137 data points
cluster 9 has 147 data points
cluster 10 has 289 data points
cluster 11 has 193 data points
cluster 12 has 56 data points
cluster 13 has 112 data points
cluster 14 has 0 data points
K=261
cluster 0 has 127 data points
cluster 1 has 508 data points
cluster 2 has 108 data points
cluster 3 has 206 data points
cluster 4 has 369 data points
cluster 5 has 379 data points
cluster 6 has 237 data points
cluster 7 has 122 data points
cluster 8 has 212 data points
cluster 9 has 74 data points
cluster 10 has 243 data points
cluster 11 has 513 data points
cluster 12 has 206 data points
cluster 13 has 186 data points
cluster 14 has 62 data points
cluster 15 has 171 data points
cluster 16 has 279 data points
cluster 17 has 445 da

In [25]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready10 = data_sorted.drop('cluster', axis=1)
print(data_ready10)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
34713  57.0068         4.59459            0            0        28.8311      0
14564  57.0068         4.59459            0            0        28.8311      0
31144  57.0068         4.59459            0            0        28.8311      0
38108  57.0068         4.59459            0            0        28.8311      1
25274  57.0068         4.59459            0            0        28.8311      0
...        ...             ...          ...          ...            ...    ...
28151  55.7165          9.5567            0            0        39.9381      0
13338  55.7165          9.5567            0            0        39.9381      0
28154  55.7165          9.5567            0            0        39.9381      1
15786  55.7165          9.5567            0            0        39.9381      0
42306  55.7165          9.5567            0            0        39.9381      0

[45222 rows x 6 columns]
Time required for perturba

In [26]:
X_train, X_test, y_train, y_test = train_test_split(data_ready10.iloc[:,0:5], np.array(data_ready10.iloc[:,5:], dtype=int), test_size=0.2)
model10 = TrainingModel((5,))
model10.fit(X_train, y_train)
model10.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77070
--------------------
Confusion Matrix:
[[6291  504]
 [1570  680]]
--------------------
              precision    recall  f1-score   support

           0       0.80      0.93      0.86      6795
           1       0.57      0.30      0.40      2250

    accuracy                           0.77      9045
   macro avg       0.69      0.61      0.63      9045
weighted avg       0.74      0.77      0.74      9045

--------------------
AUC score: 0.63627


In [27]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 5
K = int(len(new_data[0])//(k*2))
D = 500//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4522. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4522. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4522. Setting it to 3*k
  init_size=init_size)


K=4522
cluster 0 has 159 data points
cluster 1 has 78 data points
cluster 2 has 24 data points
cluster 3 has 58 data points
cluster 4 has 46 data points
cluster 5 has 46 data points
cluster 6 has 49 data points
cluster 7 has 44 data points
cluster 8 has 62 data points
cluster 9 has 7 data points
cluster 10 has 210 data points
cluster 11 has 58 data points
cluster 12 has 27 data points
cluster 13 has 55 data points
cluster 14 has 105 data points
cluster 15 has 169 data points
cluster 16 has 137 data points
cluster 17 has 124 data points
cluster 18 has 61 data points
cluster 19 has 94 data points
cluster 20 has 103 data points
cluster 21 has 137 data points
cluster 22 has 130 data points
cluster 23 has 15 data points
cluster 24 has 28 data points
cluster 25 has 235 data points
cluster 26 has 26 data points
cluster 27 has 34 data points
cluster 28 has 67 data points
cluster 29 has 45 data points
cluster 30 has 68 data points
cluster 31 has 209 data points
cluster 32 has 25 data points
clu

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4422. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4422. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4422. Setting it to 3*k
  init_size=init_size)


K=4422
cluster 0 has 53 data points
cluster 1 has 211 data points
cluster 2 has 48 data points
cluster 3 has 57 data points
cluster 4 has 212 data points
cluster 5 has 25 data points
cluster 6 has 105 data points
cluster 7 has 170 data points
cluster 8 has 30 data points
cluster 9 has 75 data points
cluster 10 has 21 data points
cluster 11 has 64 data points
cluster 12 has 54 data points
cluster 13 has 74 data points
cluster 14 has 96 data points
cluster 15 has 211 data points
cluster 16 has 9 data points
cluster 17 has 58 data points
cluster 18 has 229 data points
cluster 19 has 90 data points
cluster 20 has 106 data points
cluster 21 has 54 data points
cluster 22 has 13 data points
cluster 23 has 30 data points
cluster 24 has 201 data points
cluster 25 has 77 data points
cluster 26 has 31 data points
cluster 27 has 105 data points
cluster 28 has 102 data points
cluster 29 has 6 data points
cluster 30 has 1 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4322. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4322. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4322. Setting it to 3*k
  init_size=init_size)


...

K=522
cluster 0 has 84 data points
cluster 1 has 176 data points
cluster 2 has 72 data points
cluster 3 has 29 data points
cluster 4 has 120 data points
cluster 5 has 93 data points
cluster 6 has 165 data points
cluster 7 has 52 data points
cluster 8 has 163 data points
cluster 9 has 144 data points
cluster 10 has 8 data points
cluster 11 has 128 data points
cluster 12 has 86 data points
cluster 13 has 75 data points
cluster 14 has 138 data points
cluster 15 has 102 data points
cluster 16 has 96 data points
cluster 17 has 97 data points
cluster 18 has 121 data points
cluster 19 has 132 data points
cluster 20 has 22 data points
cluster 21 has 116 data points
cluster 22 has 89 data points
cluster 23 has 106 data points
cluster 24 has 26 data points
cluster 25 has 215 data points
cluster 26 has 105 data points
cluster 27 has 69 data points
cluster 28 has 54 data points
cluster 29 has 172 data points
cluster 30 has 40 data points
cluster 31 has 4 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=422. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=422. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=422. Setting it to 3*k
  init_size=init_size)


K=422
cluster 0 has 48 data points
cluster 1 has 169 data points
cluster 2 has 39 data points
cluster 3 has 137 data points
cluster 4 has 33 data points
cluster 5 has 171 data points
cluster 6 has 149 data points
cluster 7 has 276 data points
cluster 8 has 276 data points
cluster 9 has 108 data points
cluster 10 has 210 data points
cluster 11 has 88 data points
cluster 12 has 46 data points
cluster 13 has 114 data points
cluster 14 has 43 data points
cluster 15 has 81 data points
cluster 16 has 180 data points
cluster 17 has 157 data points
cluster 18 has 133 data points
cluster 19 has 179 data points
cluster 20 has 122 data points
cluster 21 has 28 data points
cluster 22 has 68 data points
cluster 23 has 69 data points
cluster 24 has 53 data points
cluster 25 has 102 data points
cluster 26 has 41 data points
cluster 27 has 155 data points
cluster 28 has 145 data points
cluster 29 has 223 data points
cluster 30 has 235 data points
cluster 31 has 163 data points
cluster 32 has 209 data 

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=322. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=322. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=322. Setting it to 3*k
  init_size=init_size)


K=322
cluster 0 has 277 data points
cluster 1 has 508 data points
cluster 2 has 147 data points
cluster 3 has 31 data points
cluster 4 has 52 data points
cluster 5 has 159 data points
cluster 6 has 343 data points
cluster 7 has 207 data points
cluster 8 has 156 data points
cluster 9 has 51 data points
cluster 10 has 157 data points
cluster 11 has 116 data points
cluster 12 has 168 data points
cluster 13 has 350 data points
cluster 14 has 127 data points
cluster 15 has 106 data points
cluster 16 has 208 data points
cluster 17 has 145 data points
cluster 18 has 53 data points
cluster 19 has 45 data points
cluster 20 has 227 data points
cluster 21 has 416 data points
cluster 22 has 90 data points
cluster 23 has 74 data points
cluster 24 has 151 data points
cluster 25 has 101 data points
cluster 26 has 104 data points
cluster 27 has 114 data points
cluster 28 has 236 data points
cluster 29 has 69 data points
cluster 30 has 236 data points
cluster 31 has 105 data points
cluster 32 has 151 d

In [28]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready5 = data_sorted.drop('cluster', axis=1)
print(data_ready5)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
23100  32.5075         10.2575            0            0        40.0261      0
43333  32.5075         10.2575            0            0        40.0261      0
28430  32.5075         10.2575            0            0        40.0261      0
14213  32.5075         10.2575            0            0        40.0261      0
28549  32.5075         10.2575            0            0        40.0261      1
...        ...             ...          ...          ...            ...    ...
19277  34.5223              10            0            0             40      0
5053   34.5223              10            0            0             40      1
33230  34.5223              10            0            0             40      0
4496   34.5223              10            0            0             40      1
17488  34.5223              10            0            0             40      0

[45222 rows x 6 columns]
Time required for perturba

In [29]:
X_train, X_test, y_train, y_test = train_test_split(data_ready5.iloc[:,0:5], np.array(data_ready5.iloc[:,5:], dtype=int), test_size=0.2)
model5 = TrainingModel((5,))
model5.fit(X_train, y_train)
model5.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77557
--------------------
Confusion Matrix:
[[6563  281]
 [1749  452]]
--------------------
              precision    recall  f1-score   support

           0       0.79      0.96      0.87      6844
           1       0.62      0.21      0.31      2201

    accuracy                           0.78      9045
   macro avg       0.70      0.58      0.59      9045
weighted avg       0.75      0.78      0.73      9045

--------------------
AUC score: 0.57972
